# Integrácia dát

In [8]:
from pyspark.sql import SparkSession

#### Vytvorenie SparkSession

In [9]:
spark = SparkSession.builder.appName("car_accidents").getOrCreate()
sc = spark.sparkContext
spark.sparkContext.setLogLevel("ERROR")

#### Načítanie dát

In [10]:
df_accidents = spark.read.csv("/Users/m4/Desktop/school/4/TSVD/assignment/Big-Data-Processing/data.tmp/CarAccidents/Accidents.csv", header=True, inferSchema=True)
df_casualties = spark.read.csv("/Users/m4/Desktop/school/4/TSVD/assignment/Big-Data-Processing/data.tmp/CarAccidents/Casualties.csv", header=True, inferSchema=True)
df_vehicles = spark.read.csv("/Users/m4/Desktop/school/4/TSVD/assignment/Big-Data-Processing/data.tmp/CarAccidents/Vehicles.csv", header=True, inferSchema=True)

df_weather = spark.read.csv("/Users/m4/Desktop/school/4/TSVD/assignment/Big-Data-Processing/data.tmp/AirTraffic/airport_weather_2019.csv", header=True, inferSchema=True)

df_accidents.show(5)

+--------------+---------------------+----------------------+---------+---------+------------+-----------------+------------------+--------------------+----------+-----------+-------------------+--------------------------+-------------------------+--------------+---------------+---------+-----------+---------------+----------------+--------------+---------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-------------------------+
|Accident_Index|Location_Easting_OSGR|Location_Northing_OSGR|Longitude| Latitude|Police_Force|Accident_Severity|Number_of_Vehicles|Number_of_Casualties|      Date|Day_of_Week|               Time|Local_Authority_(District)|Local_Authority_(Highway)|1st_Road_Class|1st_Road_Number|Road_Type|Speed_limit|Junction_Detail|Junction_Control|2nd_Road_Class|2nd_Road_Numbe

#### Spojenie dát podľa Accident_Index

In [11]:
df = df_accidents.join(df_casualties, df_accidents["Accident_Index"] == df_casualties["Accident_Index"], "full")
df = df.join(df_vehicles, df_accidents["Accident_Index"] == df_vehicles["Accident_Index"], "full")
df.show(5)

+--------------+---------------------+----------------------+---------+---------+------------+-----------------+------------------+--------------------+----------+-----------+-------------------+--------------------------+-------------------------+--------------+---------------+---------+-----------+---------------+----------------+--------------+---------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-------------------------+--------------+-----------------+------------------+--------------+---------------+---------------+--------------------+-----------------+-------------------+-------------------+-------------+----------------------+----------------------------------+-------------+-----------------------+--------------+-----------------+------------+-----------------------+------

#### Vytvorenie vzorky 10% dát stratifikovane

In [12]:
df_sampled = df.sampleBy("Accident_Severity", fractions={1: 0.1, 2: 0.1, 3: 0.1}, seed=1234)

In [13]:
df.groupBy("Accident_Severity").count().orderBy("Accident_Severity").show()
df_sampled.groupBy("Accident_Severity").count().orderBy("Accident_Severity").show()

+-----------------+-------+
|Accident_Severity|  count|
+-----------------+-------+
|                1|  83607|
|                2| 596571|
|                3|3607415|
+-----------------+-------+



+-----------------+------+
|Accident_Severity| count|
+-----------------+------+
|                1|  8145|
|                2| 59787|
|                3|360841|
+-----------------+------+



#### Rozdelenie dát na trénovaciu a testovaciu množinu

In [14]:
df_train, df_test = df_sampled.randomSplit([0.70, 0.30], seed=1234)

df_train.groupBy("Accident_Severity").count().orderBy("Accident_Severity").show()
df_test.groupBy("Accident_Severity").count().orderBy("Accident_Severity").show()

+-----------------+------+
|Accident_Severity| count|
+-----------------+------+
|                1|  5729|
|                2| 41756|
|                3|252817|
+-----------------+------+



+-----------------+------+
|Accident_Severity| count|
+-----------------+------+
|                1|  2494|
|                2| 17814|
|                3|107973|
+-----------------+------+

